In [ ]:
#该代码用于对石笋数据进行定距离的插值，使之与降水数据有一些相同的年份。
#将这些相同年份进行一一对应，对石笋和所有站点的降水一个个进行相关分析，并生成列表

import math
import pandas as pd
import scipy.stats
import numpy as np

def coordinate(x):
    degree = math.floor(x)
    minutes = round(((x - degree) * 100)/60,2)
    result = degree + minutes
    return result

pointsname = pd.read_excel('point(去除东山).xls', index_col=None, na_values=['NA'])
pointnumber = pointsname.loc[:,'区站号'].tolist()
pointname = pointsname.loc[:,'站名'].tolist()
pointprovince = pointsname.loc[:,'省份'].tolist()
pointlatitude = pointsname.loc[:,'纬度（度.分）'].map(lambda x: coordinate(x)).tolist()
pointlongitude = pointsname.loc[:,'经度（度.分）'].map(lambda x: coordinate(x)).tolist()
pointaltitude = pointsname.loc[:,'观测场拔海高度（米）'].tolist()
numbername = dict(zip(pointnumber, pointname))
numberprovince = dict(zip(pointnumber, pointprovince))
numberlatitude = dict(zip(pointnumber, pointlatitude))
numberlongitude = dict(zip(pointnumber, pointlongitude))
numberaltitude = dict(zip(pointnumber, pointaltitude))

In [ ]:
def produceTimeseries(D):
    startdate = 1951
    timeseries=[]
    timeseries.append(startdate)
    nowdate = startdate + D
    while nowdate <= 2019:
        timeseries.append(nowdate)
        nowdate = nowdate + D
    timeresult= pd.DataFrame({'Timeseries':timeseries[::-1]}) 
    return timeresult


def interporlateseries(data, timeseires):
    targertime = timeseires.iloc[:, 0]
    time = data.iloc[:, 0]
    calculatedata = data.iloc[:, 1]
    
    interporateresult = []
    interporatetime = []
    for neartime in range(0, len(targertime)):
        if pd.isnull(targertime.iloc[neartime]) == False:            
            for nearx in range(0,len(time)):
                if pd.isnull(time.iloc[nearx])  == False:
                    if targertime.iloc[neartime] < time.iloc[nearx] and targertime.iloc[neartime] > time.iloc[nearx + 1]:
                        xp = [time.iloc[nearx + 1], time.iloc[nearx]]
                        yp = [calculatedata.iloc[nearx + 1], calculatedata.iloc[nearx]]
                        interporatetime.append (targertime[neartime])
                        interporateresult.append (round(np.interp(targertime[neartime],xp, yp),4))
                    elif targertime.iloc[neartime] == time.iloc[nearx]:
                        interporatetime.append (time.iloc[nearx])
                        interporateresult.append (round(calculatedata.iloc[nearx],4))
                
    interresult = pd.DataFrame({data.columns[0]: interporatetime,
                                data.columns[1]: interporateresult})
    return interresult

In [ ]:
def correlate2series(stalgdata,rainfalldata,lagmax): 
    
    result = pd.DataFrame()
    
    for point in pointnumber:
        
        resultpoint = []
        resultname = []
        resultprovince = []
        resultlatitude = []
        resultlongitude = []
        resultaltitude = []
                
        resultpoint.append (point)
        resultname.append (numbername.get(point))
        resultprovince.append (numberprovince.get(point))
        resultlatitude.append(numberlatitude.get(point))
        resultlongitude.append(numberlongitude.get(point))
        resultaltitude.append(numberaltitude.get(point))

            
        dataresult = pd.DataFrame({'PointNumber': resultpoint,
                                    'PointName': resultname,
                                    'PointProvince':resultprovince,
                                    'Latitude': resultlatitude,
                                    'Longitude': resultlongitude,
                                    'Altitude':resultaltitude})
        
        try:
            forcompare = pd.DataFrame({'唯一标示': rainfalldata.loc[:,'唯一标示'],
                                        numbername.get(point): rainfalldata.loc[:,numbername.get(point)]})
            precorr = pd.merge(stalgdata, forcompare, on='唯一标示') 
            precorr= precorr.dropna(axis=0, how='any')
        except:
            pass
        finally:
            print(numbername.get(point))
            print(precorr.shape[0])        
        
        if precorr.shape[0]> 0:
            
            for lagyear in list(range(0,lagmax+1)): 
            
                    resultcorr = []
                    resultp = []
                    resultn = []
                
                    timeofStalg = stalgdata.loc[:,'唯一标示']
                    dataofStalg = stalgdata.loc[:,'石笋']
                
                    timeofStalg = timeofStalg.drop(timeofStalg.index[0:lagyear]).reset_index(drop=True)
                    dataofStalg = dataofStalg.drop(dataofStalg.index[dataofStalg.shape[0]-lagyear:dataofStalg.shape[0]]).reset_index(drop=True)
                
                    lagseriesofStalg = pd.concat([timeofStalg,dataofStalg], axis=1)
                    forcompareinlag = pd.DataFrame({'唯一标示': precorr.loc[:,'唯一标示'],
                                                    numbername.get(point): precorr.loc[:,numbername.get(point)]})             
                    precorrinlag = pd.merge(lagseriesofStalg, forcompareinlag, on='唯一标示').dropna(axis=0, how='any')          
                     
                    
                    print(precorrinlag.shape[0])     
                    
                    
                    if precorrinlag.shape[0] > 3:
                        correlateresult = scipy.stats.pearsonr(precorrinlag.iloc[:,1], precorrinlag.iloc[:,2])
                
                        resultcorr.append (round(correlateresult[0],4))
                        resultp.append(round(correlateresult[1],4))
                        resultn.append(len(precorr))    

                
                        dataresultinlag = pd.DataFrame({'R_Lag_'+ str(lagyear) + '_unit': resultcorr,
                                                        'P_Lag_'+ str(lagyear) + '_unit': resultp,
                                                        'N_Lag_'+ str(lagyear) + '_unit': resultn})

                        dataresult = pd.concat([dataresult,dataresultinlag], axis=1)
                        

        result= pd.concat([result, dataresult],axis=0)
    
    distance = stalgdata.iloc[0,0] - stalgdata.iloc[1,0]
    startyear = stalgdata.iloc[0,0]
    endyear = stalgdata.iloc[-1,0] 
    config = pd.DataFrame({'Name':['Distance','Startyear','Endyear','Lag'],
                          'Data':[distance,startyear,endyear,lagmax]})
    
    return [result,config]

In [ ]:
Filename = 'LFD_1.xlsx' 
stalagdata = pd.read_excel(Filename,  index_col=None, na_values=['NA'])
stalagdataforcal = interporlateseries(data=stalagdata,timeseires=produceTimeseries(2))

rainfalldata = pd.read_excel('全国降水结果(季节)(修正版).xlsx','summer_year', index_col=None, na_values=['NA'])

process = correlate2series(stalgdata=stalagdataforcal,rainfalldata=rainfalldata ,lagmax=7)

Selecteddatawriter = pd.ExcelWriter('corrStalg_Rainfall_'+Filename)
process[0].to_excel(Selecteddatawriter,index=False, sheet_name='Data') 
process[1].to_excel(Selecteddatawriter,index=False, sheet_name='Config') 
Selecteddatawriter.save()

print (process[0])